In [1]:
import os,sys
sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..')))
import warnings
warnings.filterwarnings("ignore")

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import CPS

In [3]:
opt = CPS.config()
args = opt.parse_args(['--seed', '2025'])
args.prep_scale = False
args.sh_weights = False
args.dataset_path = '/mnt/d/Dataset/SRT_Dataset/1-DLPFC/'
CPS.set_random_seed(args.seed)
args

Namespace(batch_size=256, clusters=7, coord_dim=2, dataset_path='/mnt/d/Dataset/SRT_Dataset/1-DLPFC/', decoder='MLP', decoder_latent=[256, 512], distill=0.5, dropout=0.2, flow='source_to_target', freq=32, gpu=0, hvgs=3000, inr_latent=[256, 256, 256], k_list=[0, 1, 2, 3, 4, 5, 6, 7], latent_dim=64, lr=0.001, max_epoch=200, max_neighbors=6, n_spot=0, num_heads=8, prep_scale=False, radius=150, seed=2025, self_loops=True, sh_weights=False, sigma=10.0, visual=True, weight_decay=0.0001)

In [4]:
section = '151673'
adata = sc.read_visium(os.path.join(args.dataset_path+section))
Ann_df = pd.read_csv(os.path.join(args.dataset_path+'1-DLPFC_annotations/', section+'_truth.txt'), 
                     sep='\t', header=None, index_col=0)
Ann_df.columns = ['Ground Truth']
adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth']
adata

AnnData object with n_obs × n_vars = 3639 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'Ground Truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [19]:
adata.obs['in_tissue']

AAACAAGTATCTCCCA-1    1
AAACAATCTACTAGCA-1    1
AAACACCAATAACTGC-1    1
AAACAGAGCGACTCCT-1    1
AAACAGCTTTCAGAAG-1    1
                     ..
TTGTTTCACATCCAGG-1    1
TTGTTTCATTAGTCTA-1    1
TTGTTTCCATACAACT-1    1
TTGTTTGTATTACACG-1    1
TTGTTTGTGTAAATTC-1    1
Name: in_tissue, Length: 3639, dtype: int64

In [11]:
adata.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC233755.2,ENSG00000277856,Gene Expression,GRCh38
AC233755.1,ENSG00000275063,Gene Expression,GRCh38
AC240274.1,ENSG00000271254,Gene Expression,GRCh38
AC213203.1,ENSG00000277475,Gene Expression,GRCh38


In [6]:
adata.obsm['spatial']

array([[ 9791,  8468],
       [ 5769,  2807],
       [ 4068,  9505],
       ...,
       [ 4631,  7831],
       [ 5571, 11193],
       [ 6317,  3291]])

In [9]:
adata.uns

OrderedDict([('spatial',
              {'151673': {'images': {'hires': array([[[0.68235296, 0.69803923, 0.69411767],
                         [0.68235296, 0.69411767, 0.6901961 ],
                         [0.6784314 , 0.69803923, 0.6901961 ],
                         ...,
                         [0.6784314 , 0.6901961 , 0.6901961 ],
                         [0.6745098 , 0.69411767, 0.6901961 ],
                         [0.68235296, 0.69803923, 0.6901961 ]],
                 
                        [[0.68235296, 0.69411767, 0.6901961 ],
                         [0.68235296, 0.69803923, 0.6862745 ],
                         [0.68235296, 0.69411767, 0.6862745 ],
                         ...,
                         [0.6745098 , 0.6862745 , 0.6862745 ],
                         [0.6784314 , 0.69411767, 0.6901961 ],
                         [0.68235296, 0.69411767, 0.6901961 ]],
                 
                        [[0.6862745 , 0.69411767, 0.6862745 ],
                         [0.6